<a href="https://colab.research.google.com/github/JairsonAlbertoSami/desafio-junior-data-scientis/blob/main/Analise_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Técnico - Cientista de Dados Júnior - Rio de Janeiro

Este código foi desenvolvido como parte do desafio técnico para a posição de **Cientista de Dados Júnior** no **Rio de Janeiro**. O objetivo deste desafio é demonstrar habilidades em manipulação e análise de dados python e integração com APIs.


## Integração com APIs: Feriados e Tempo

### Utilize as APIs públicas abaixo para responder às questões 1-8:
- [Public Holiday API](https://date.nager.at/Api)
- [Open-Meteo Historical Weather API](https://open-meteo.com/)


In [2]:
!pip install requests

In [3]:
import requests
import datetime
import json

# 1.  **Quantos feriados há no Brasil em todo o ano de 2024?**